In [2]:
import scipy
import os,sys,csv
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
from scipy.signal import medfilt,find_peaks

# %%%

In [11]:
def data_processing(data_loc, file_loc):
    files = os.listdir(data_loc)
    print("total files : {}".format(len(files)))
    Data = []

    for name in files:
        full_path = os.path.join(data_loc, name)
        file_data = pd.read_csv(full_path, sep=" ", header = None)
        df = pd.DataFrame(file_data)

        ##.......Reducing Baseline from 365 to 0.......##
    
        MLII = list(df[5])
        baseline = medfilt(MLII, 71) 
        baseline = medfilt(baseline, 215) 

        # Remove Baseline
        for i in range(0, len(MLII)):
            MLII[i] = MLII[i] - baseline[i]
    
        ##.....Renovel of High Freq. Noise.....##
    
        df[5] = scipy.signal.savgol_filter(MLII, window_length = 7, polyorder = 2, deriv=0, delta=1.0, axis=-1, mode='interp', cval=0.0)
        df[5] = (df[5]-df[5].min())/(df[5].max()-df[5].min())
        
        
        peak_index=find_peaks(df[5],height=[0.8,1])[0]
        
        for i in range (min(len(peak_index),100)-1):
        
            if(name[0] == 'p'):
                label=1
            else:
                label=0
            
            
            d = df[5].values.tolist()[peak_index[i]:peak_index[i+1]]
            remain=187-len(d)
            if(remain>0):
                d.extend([0]*remain)
            else:
                d=d[:187]
            d.append(label)
            Data.append(d)
            
    my_df = pd.DataFrame(Data)
    my_df.to_csv(file_loc, encoding='utf-8', index=False)

# %%%

In [12]:
patient = 'heartbeat/ecg_patient.csv'
normal = 'heartbeat/ecg_normal.csv'

In [13]:
fold_name = 'heartbeat/ecg_normal'
print('Data Processing for normal ............')
data_processing(fold_name, normal)

fold_name = 'heartbeat/ecg_patient'
print('Data Processing for patient ............')
data_processing(fold_name, patient)

Data Processing for normal ............
total files : 247
Data Processing for patient ............
total files : 53


## data for anomaly detection

In [14]:
df_patient=pd.read_csv("heartbeat/ecg_patient.csv")
df_normal=pd.read_csv("heartbeat/ecg_normal.csv")

In [15]:
df_patient.shape,df_normal.shape

((2900, 188), (22365, 188))

In [6]:
df_normal_train=df_normal.iloc[:4000]
df_normal_valid=df_normal.iloc[4000:4120]
df_patient=df_patient.iloc[:120]

In [7]:
df_normal_train.to_csv('./input/anomaly/inhouse/normal_train.csv', encoding='utf-8', index=False)
df_normal_valid.to_csv('./input/anomaly/inhouse/normal_valid.csv', encoding='utf-8', index=False)
df_patient.to_csv('./input/anomaly/inhouse/patient_valid.csv', encoding='utf-8', index=False)

## data for classification

In [16]:
df_patient=pd.read_csv("heartbeat/ecg_patient.csv")
df_normal=pd.read_csv("heartbeat/ecg_normal.csv")

In [17]:
df_patient.shape,df_normal.shape

((2900, 188), (22365, 188))

In [18]:
df_train=np.vstack((df_patient.values[:2800],df_normal.values[:3000]))
df_train=pd.DataFrame(df_train)
df_train = df_train.sample(frac=1).reset_index(drop=True)

df_test=np.vstack((df_patient.values[2800:2900],df_normal.values[3000:3100]))
df_test=pd.DataFrame(df_test)
df_test = df_test.sample(frac=1).reset_index(drop=True)

print(df_train.shape,df_test.shape)
df_train.to_csv("./input/classification/inhouse/train.csv",index=False)
df_test.to_csv("./input/classification/inhouse/test.csv",index=False)

(5800, 188) (200, 188)
